# USING OLLAMA

---

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [14]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Descrble what Akamai does"}
]

In [15]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [9]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [10]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Here is the translation:

```
あらためて、ありがとうございました。
```

Or, in a more casual tone:

```
すごくありがとう
```


# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [11]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Here is the translation:

「ありがとう」 (Arigatou)

Or, in a more polite and formal tone:

「ありがとうございます」 (Arigatou gozaimasu)

The latter is used when expressing gratitude or appreciation for something. The former is a more casual way of saying "thank you".


## Alternative approach - using OpenAI python library to connect to Ollama

In [16]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Akamai Technologies is a leading provider of cloud-based service solutions for accelerating and securing online content delivery. Here's an overview of what they do:

**Key Services:**

1. **Content Delivery Network (CDN)**: Akamai's Content Delivery Network ( CDN) is a global network of servers strategically located across the world. The CDN acts as a proxy between users' devices and the origin server, caching frequently accessed content at edge locations closer to users. This reduces latency, improves performance, and ensures high availability.
2. **Security Services**: Akamai offers advanced security solutions, including web applicationFirewall (WAF), bot mitigation, and DDoS protection. Their services help protect websites, web applications, and APIs from cyber threats like malware, SQL injection, and cross-site scripting (XSS) attacks.
3. **CDN + Security Integration**: Akamai's full-service CDN+Security allows organizations to combine their CDNs with security features, providing 

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

<think>
Okay, I need to explain the key concepts behind Large Language Models (LLMs), specifically focusing on what neural networks are, attention mechanisms in transformers, and maybe something about how transformers themselves work. Let me break this down step by step.

First, understanding what a neural network is. Neural networks are computational models inspired by the structure of the human brain. They consist of layers, nodes (or neurons) connected together with weights that represent connections or dependencies. So these models process information through layers where each layer transforms the inputs into representations that can be used for the next layer. Think of it as a way to model complex patterns in data.

Next, attention mechanisms in transformers. I remember transformers are known for their efficiency and effectiveness in NLP tasks. The key part here is how they manage the flow of information. The mechanism called " attended self-attention" allows each token in a seque